In [5]:
import argparse
from utils.load_params import load_params
from utils.sample import sample
from model.train import train
from dataset.mnist_dataset import MnistDataset
from torch.utils.data import DataLoader
from noise_scheduler.scheduler import NoiseScheduler
import torch
from model.unet import Unet

from noise_scheduler._archived_scheduler import NoiseScheduler as NoiseScheduler2

### Load Parameters
The parameters are available under `config.json`

In [6]:
#parser = argparse.ArgumentParser(description="Run models with optional training.")
#args = parser.parse_args()
params = load_params("config.json")

### Load Data

In [7]:
traindataset = MnistDataset(root_dir=params["root_dir"])
train_loader = DataLoader(traindataset, batch_size=params["batch_size"], shuffle=True, num_workers=2)

Found 60000 images


### Noise Scheduler

In [ ]:
train_model = False
if train_model:
    device= 'cuda' if torch.cuda.is_available() else 'cpu'  
    scheduler = NoiseScheduler(device, params["num_timesteps"], params["beta_start"], params["beta_end"])
    train(
        epochs_num=params["epochs_num"],
        train_loader=train_loader,
        noise_scheduler=scheduler
    )
else:
    scheduler = NoiseScheduler2(params["num_timesteps"], params["beta_start"], params["beta_end"])
    model = Unet()
    state_dict = torch.load("model/model.pth", map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    sample(
        model=model,
        scheduler=scheduler,
        config=params
    )
    

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.